### SVD - kompresja obrazu

In [ ]:
from matplotlib.image import imread
import matplotlib.pyplot as plt
import numpy as np
import os
# plt.rcParams['figure.figsize'] = [16, 8]


A = imread(os.path.join('..','Dane','Obrazy','dog.jpg'))
X = np.mean(A, -1); # Convert RGB to grayscale
print(X.shape)

img = plt.imshow(X)
img.set_cmap('gray')
plt.axis('off')
plt.show()


In [ ]:
U, S, VT = np.linalg.svd(X,full_matrices=False)
S = np.diag(S)

j = 0
for r in (5, 20, 100):
    # Construct approximate image
    Xapprox = U[:,:r] @ S[0:r,:r] @ VT[:r,:]
    plt.figure(j+1)
    j += 1
    img = plt.imshow(Xapprox)
    img.set_cmap('gray')
    plt.axis('off')
    plt.title('r = ' + str(r))
    plt.show()

In [ ]:
plt.figure(1)
plt.semilogy(np.diag(S))  # ile energii tłumaczy wartość
plt.title('Singular Values')
plt.show()

plt.figure(2)
plt.plot(np.cumsum(np.diag(S))/np.sum(np.diag(S)))  # ile energii przetłumaczone
plt.title('Singular Values: Cumulative Sum')
plt.show()

### Zaprojektować wyszukiwarkę: dla danego zapytania znaleźć tekst najbardziej adekwatny.

Podstawowy sposób reprezentacji tekstu to macierz wystąpień słów w dokumentach - oznacza to bardzo duży wymiar danych...

Dlatego potrzebna redukcja. Pomysł: wykryć znaczenie tekstu znajdując ukryte cechy.

W praktyce rozkład SVD stosuje się w analizie podobieństwa tekstów (wyszukiwarki, systemy rekomendacyjne).

In [ ]:
documents = [
    "morze plaża woda słońce piasek",
    "morze plaża piasek woda",
    "morze plaża słońce",
    "plaża woda piasek",
    "słońce woda plaża",
    "piasek woda",
    "krowa pies papuga",
    "pies kot"
]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 
cv = CountVectorizer()
dtm = cv.fit_transform(documents) 
dtm 

In [ ]:
x = dtm.todense().T
x

In [ ]:
x = dtm.todense().T 
x  # wiersze słowa, kolumny - dokumenty

Spróbujmy zareprezentować dokumenty w niżej wymiarowej przestrzeni

In [ ]:
U, s, V = np.linalg.svd(x)
S = np.zeros_like(x).astype("float")
np.fill_diagonal(S,s)

In [ ]:
np.dot(np.dot(U,S),V)

In [ ]:
# Ile wziac skladowych?
S 

In [ ]:
k = 3
np.set_printoptions(2,suppress=True)

x_n = V[:k,:] # - jedna kolumna reprezentuje jeden dokument (w mniejszej przestrzeni) 
#                 zatem bierzemy pierwsze k współrzędnych z każdego dokumentu
x_n

Spójrzmy jak otrzymać V z X mając rozkład svd:

$ X = USV^t \ \ $ => $ \ \ U^{-1} X = SV^t \ \ $ => $ \ \ S^{-1} U^{-1} X = V^t$ 

i wiemy, że $U^{-1} = U^t$.

Zatem co zrobić z nowym zapytanie, żeby otrzymać jego reprezentację w V? To samo co z X, czyli:

$ V^t_{nowe} = S^{-1} U^t X_{nowy}$

In [ ]:
#Zatem, aby otrzymać k-wymiarową reprezentację V z X:
dk = np.dot(np.linalg.inv(S[:k,:k]), np.dot(U.T[:k,:],x))
dk

In [ ]:
zapytanie = ["morze i słońce"]
zapytanie = cv.transform(zapytanie).todense().T # reprezentujemy zapytanie jak wektor wystapień słów
zapytanie

In [ ]:
#Zatem reprezentacja zapytania w przestrzeni dokumentów:
q = np.dot(np.linalg.inv(S[:k,:k]), np.dot(U.T[:k,:],zapytanie))
q = np.asarray(q)
q

In [ ]:
dk=np.asarray(dk)

In [ ]:
from sklearn.metrics import pairwise_distances
pairwise_distances(dk.T,q.T)

In [ ]:
kolejnosc = pairwise_distances(dk.T,q.T).flatten().argsort()
kolejnosc

In [ ]:
for item in np.array(documents)[kolejnosc]:
    print(item)